In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import datetime
from datetime import date
from datetime import timedelta
import statsmodels.formula.api as smf
from scipy.stats import gaussian_kde
from scipy.stats import norm
import statistics
from scipy import stats
from scipy.optimize import minimize
from sklearn.model_selection import KFold
from matplotlib.colors import LinearSegmentedColormap
import seaborn as sns
from matplotlib.patches import Rectangle
import os
import re
from datetime import datetime
import pytz
from sklearn.model_selection import TimeSeriesSplit
import smtplib
from email.mime.text import MIMEText
from email.mime.multipart import MIMEMultipart
import glob

In [2]:

# Load the dataset
cdf = pd.read_csv('nfl_outcomes.csv')

# Convert 'game_date' column to datetime format
cdf['game_date'] = pd.to_datetime(cdf['game_date'])

# Compute margin of victory
cdf['margin'] = cdf['total_home_score'] - cdf['total_away_score']

cdf.rename(columns={'home_team': 'home', 'away_team': 'vis'}, inplace=True)

# Sort the DataFrame by year and week to maintain chronological order
cdf = cdf.sort_values(by=['year', 'week']).reset_index(drop=True)
cdf = cdf.drop(columns=["game_id"])


# Generate the Elo dictionary with all teams
teams = pd.concat([cdf['home'], cdf['vis']]).unique()
elo_dict = {team: 1500 for team in teams}

cdf.tail(30)

,game_date,year,week,home,vis,home_qb,away_qb,home_qb_epa,away_qb_epa,home_qb_snaps,away_qb_snaps,home_def_epa,away_def_epa,total_home_score,total_away_score,margin
6673,2024-12-26,2024,17,CHI,SEA,C.Williams,G.Smith,-0.246960,-0.227852,38,28,0.227852,0.246960,3.0,6.0,-3.0
6674,2024-12-29,2024,17,JAX,TEN,M.Jones,M.Rudolph,0.373519,0.019044,26,38,-0.019044,-0.373519,20.0,13.0,7.0
6675,2025-01-05,2024,18,NE,BUF,J.Milton,M.Trubisky,0.430503,0.215088,31,22,-0.215088,-0.430503,23.0,16.0,7.0
6676,2025-01-05,2024,18,ATL,CAR,M.Penix,B.Young,0.202173,0.605024,40,38,-0.605024,-0.202173,38.0,44.0,-6.0
6677,2025-01-05,2024,18,GB,CHI,M.Willis,C.Williams,-0.162455,-0.021404,17,31,0.021404,0.162455,22.0,24.0,-2.0
6678,2025-01-04,2024,18,PIT,CIN,R.Wilson,J.Burrow,-0.124051,-0.028525,37,50,0.028525,0.124051,17.0,19.0,-2.0
6679,2025-01-04,2024,18,BAL,CLE,L.Jackson,B.Zappe,0.149284,-0.519353,37,33,0.519353,-0.149284,35.0,10.0,25.0
6680,2025-01-05,2024,18,TEN,HOU,W.Levis,D.Mills,0.201639,-0.328062,20,24,0.328062,-0.201639,14.0,23.0,-9.0
6681,2025-01-05,2024,18,IND,JAX,J.Flacco,M.Jones,-0.085930,0.100491,43,36,-0.100491,0.085930,26.0,23.0,3.0
6682,2025-01-05,2024,18,DEN,KC,B.Nix,C.Wentz,0.816588,-0.418327,33,21,0.418327,-0.816588,38.0,0.0,38.0


In [3]:
# Function to generate training-validation folds 
# I just use 5 fold CV but hard to delete this function without disrupting the code
def generate_time_periods(max_folds=8):
    time_periods_dict = {}

    for num_folds in range(5, max_folds + 1):  # 3 to max_folds (default 8) 
        train_periods = []
        total_train_years = 2019  # Training should go up to 2019 only
        val_size = 3  # Validation period is always 4 years

        for i in range(num_folds):
            train_start = 2000 + i  # Training expands year by year
            train_end = total_train_years - (num_folds - (i + 1)) - val_size
            val_start = train_end + 1
            val_end = val_start + val_size - 1

            if val_end <= 2019:  # Ensure validation stays within range
                train_periods.append((train_start, train_end, val_start, val_end))

        time_periods_dict[num_folds] = train_periods  # Store for each fold size

    return time_periods_dict

# === Elo Computation with Team & QB Elo Updates ===
def compute_MSE_and_update_elo(params, cdf, initial_elo, initial_qb_elo):
    """
    Compute MSE and update Elo ratings for teams and QBs dynamically while optimizing parameters.
    
    Parameters:
    - params: List of parameters to optimize (k_one, sigma_one, hfa, team_mean_revert, qb_mean_revert, k_qb).
    - cdf: DataFrame containing game data.
    - initial_elo: Dictionary of initial Elo ratings for each team.
    - initial_qb_elo: Dictionary of initial QB Elo ratings.
    
    Returns:
    - MSE: Mean squared error of predicted vs. actual game margins.
    - Updated team Elo dictionary.
    - Updated QB Elo dictionary.
    """
    # Unpack parameters
    k_one, sigma_one, hfa, team_mean_revert, qb_mean_revert, k_qb = params
    
    # Copy initial Elo ratings
    elo_dict = initial_elo.copy()
    qb_elo_dict = initial_qb_elo.copy()
    
    # Initialize MSE
    mse = 0
    
    # Track the current year for applying mean reversion
    current_year = cdf.iloc[0]['year']
    
    for _, row in cdf.iterrows():
        home = row['home']
        vis = row['vis']
        home_qb = row['home_qb']
        away_qb = row['away_qb']
        margin = row['margin']
        home_qb_epa = row['home_qb_epa']
        away_qb_epa = row['away_qb_epa']
        year = row['year']
        
        # Apply mean reversion at the start of a new year
        # Apply mean reversion at the start of a new year
        if year != current_year:
            current_year = year
    
            # Apply mean reversion for teams
            for team in elo_dict:
                elo_dict[team] -= team_mean_revert * (elo_dict[team] - 1500)
    
    # Apply mean reversion for QBs (only for those who played in the past but not this year)
            for qb in list(qb_elo_dict.keys()):  
                qb_elo_dict[qb] -= qb_mean_revert * (qb_elo_dict[qb] - 1500)  # Gradual decay to 1500

        
        # Get current team and QB Elo ratings
        elo_home = elo_dict[home]
        elo_vis = elo_dict[vis]
        qb_elo_home = qb_elo_dict.get(home_qb, 1400)
        qb_elo_vis = qb_elo_dict.get(away_qb, 1400)
        
        # **Make prediction using current QB Elo (before updating)**
        elo_home_adj = elo_home + (qb_elo_home - 1500)
        elo_vis_adj = elo_vis + (qb_elo_vis - 1500)
        predicted_margin = abs(elo_home_adj + hfa - elo_vis_adj) / sigma_one
        #predicted_margin = (abs(elo_home_adj + hfa - elo_vis_adj) + (abs(elo_home_adj + hfa - elo_vis_adj) ** sigma_one)) / 100
        if elo_vis_adj > elo_home_adj + hfa:
            predicted_margin *= -1
        
        # Compute squared error for MSE calculation
        mse += (margin - predicted_margin) ** 2
        
        # **Update Team Elo ratings**
        update = k_one * (margin - predicted_margin)
        elo_dict[home] += update
        elo_dict[vis] -= update
        
        # **Compute QB performance deviation AFTER the prediction**
        expected_qb_epa_home = (qb_elo_home - 1500) / 1000 - 0.05
        expected_qb_epa_vis = (qb_elo_vis - 1500) / 1000 - 0.05

        qb_performance_delta_home = home_qb_epa - expected_qb_epa_home
        qb_performance_delta_vis = away_qb_epa - expected_qb_epa_vis

        # **Now update QB Elo ratings**
        qb_elo_home += k_qb * qb_performance_delta_home
        qb_elo_vis += k_qb * qb_performance_delta_vis
        
        # Store updated QB Elo
        qb_elo_dict[home_qb] = qb_elo_home
        qb_elo_dict[away_qb] = qb_elo_vis
    
    return mse, elo_dict, qb_elo_dict  # Return MSE, team Elo, and QB Elo

# === Cross-Validation Function ===
def cross_validation_objective(params, cdf, time_periods):
    """
    Objective function for cross-validation.
    Computes the average MSE across all time periods (folds).
    """
    total_mse = 0

    for train_start, train_end, test_start, test_end in time_periods:
        # Split dataset into training and testing periods
        train_df = cdf[(cdf['year'] >= train_start) & (cdf['year'] <= train_end)]
        test_df = cdf[(cdf['year'] >= test_start) & (cdf['year'] <= test_end)]
        
        # Initialize Elo ratings to 1500
        initial_elo = {team: 1500 for team in pd.concat([train_df['vis'], train_df['home']]).unique()}
        initial_qb_elo = {qb: 1400 for qb in pd.concat([train_df['home_qb'], train_df['away_qb']]).unique()}

        # Train the Elo function
        _, final_elo_dict, final_qb_elo_dict = compute_MSE_and_update_elo(params, train_df, initial_elo, initial_qb_elo)
        
        # Evaluate on the test set
        mse, _, _ = compute_MSE_and_update_elo(params, test_df, final_elo_dict, final_qb_elo_dict)
        total_mse += mse

    return total_mse / len(time_periods)



# Generate training-validation folds
time_periods_dict = generate_time_periods(max_folds=5)

# Define the final test set (2021-2024)
test_df = cdf[(cdf['year'] >= 2020) & (cdf['year'] <= 2024)]

# Store results for different fold numbers
results = {}

for num_folds, time_periods in time_periods_dict.items():
    print(f"\n=== Running {num_folds}-Fold Cross-Validation ===")

    # Optimize K, Sigma, HFA, Team Mean Revert, QB Mean Revert, K_QB using L-BFGS-B
    initial_params = [2.28, 50, 109, 0.47, 0.1, 100]  # Initial values
    bounds = [(0, 20), (0, 100), (0, 200), (0, 1), (0, 1), (0, 200)]  # Parameter bounds

    result = minimize(
        lambda params: cross_validation_objective(params, cdf, time_periods),
        x0=initial_params,
        bounds=bounds,
        method='L-BFGS-B'
    )

    # Extract optimized parameters
    best_k, best_sigma, best_hfa, best_team_mean_revert, best_qb_mean_revert, best_k_qb = result.x

    # Train Elo using the best params on all pre-2021 data
    full_train_df = cdf[(cdf['year'] < 2020)]
    initial_elo = {team: 1500 for team in pd.concat([full_train_df['vis'], full_train_df['home']]).unique()}
    initial_qb_elo = {qb: 1400 for qb in pd.concat([full_train_df['home_qb'], full_train_df['away_qb']]).unique()}

    _, final_elo_dict, final_qb_elo_dict = compute_MSE_and_update_elo(
        [best_k, best_sigma, best_hfa, best_team_mean_revert, best_qb_mean_revert, best_k_qb],
        full_train_df, initial_elo, initial_qb_elo
    )

    # Evaluate the best model on the test set (2021-2024)
    test_mse, final_elo_dict, final_qb_elo_dict = compute_MSE_and_update_elo(
        [best_k, best_sigma, best_hfa, best_team_mean_revert, best_qb_mean_revert, best_k_qb],
        test_df, final_elo_dict, final_qb_elo_dict
    )

    # Store results
    results[num_folds] = {
        'MSE': test_mse,
        'params': (best_k, best_sigma, best_hfa, best_team_mean_revert, best_qb_mean_revert, best_k_qb),
        'team_elo': final_elo_dict,
        'qb_elo': final_qb_elo_dict
    }

    print(f"Optimized K: {best_k:.4f}, Sigma: {best_sigma:.4f}, HFA: {best_hfa:.4f}, "
          f"Team Mean Revert: {best_team_mean_revert:.4f}, QB Mean Revert: {best_qb_mean_revert:.4f}, K_QB: {best_k_qb:.4f}")
    print(f"Test Set MSE (2020-2024): {test_mse:.4f}")

# Find the best model based on test MSE
best_folds = min(results, key=lambda x: results[x]['MSE'])
best_params = results[best_folds]['params']
best_team_elo = results[best_folds]['team_elo']
best_qb_elo = results[best_folds]['qb_elo']

print(f"\nBest Fold Count: {best_folds}")
print(f"Best Parameters: K={best_params[0]}, Sigma={best_params[1]}, HFA={best_params[2]}, "
      f"Team Mean Revert={best_params[3]}, QB Mean Revert={best_params[4]}, K_QB={best_params[5]}")
print(f"Best Test MSE (2020-2024): {results[best_folds]['MSE']:.4f}")

# Print final team Elo ratings (sorted) - NOW REPRESENTING THE MOST RECENT ELO
sorted_team_elo = sorted(best_team_elo.items(), key=lambda x: x[1], reverse=True)
print("\nFinal Team Elo Ratings (Top 10) - Current Elo:")
for team, elo in sorted_team_elo[:10]:
    print(f"{team}: {elo:.2f}")

# Print final QB Elo ratings (sorted) - NOW REPRESENTING THE MOST RECENT ELO
sorted_qb_elo = sorted(best_qb_elo.items(), key=lambda x: x[1], reverse=True)
print("\nFinal QB Elo Ratings (Top 10) - Current Elo:")
for qb, elo in sorted_qb_elo[:10]:
    print(f"{qb}: {elo:.2f}")


=== Running 5-Fold Cross-Validation ===
Optimized K: 2.0980, Sigma: 52.0968, HFA: 113.7383, Team Mean Revert: 0.4281, QB Mean Revert: 0.0000, K_QB: 102.8802
Test Set MSE (2020-2024): 236839.4025

Best Fold Count: 5
Best Parameters: K=2.0979945590794187, Sigma=52.09675297091032, HFA=113.73826720473153, Team Mean Revert=0.4280708251592234, QB Mean Revert=0.0, K_QB=102.88018807585571
Best Test MSE (2020-2024): 236839.4025

Final Team Elo Ratings (Top 10) - Current Elo:
PHI: 1805.77
BAL: 1757.20
DET: 1740.02
BUF: 1725.55
DEN: 1679.52
GB: 1660.71
TB: 1598.28
MIN: 1592.47
KC: 1592.43
PIT: 1585.23

Final QB Elo Ratings (Top 10) - Current Elo:
L.Jackson: 1902.65
J.Allen: 1824.16
B.Mayfield: 1817.10
J.Goff: 1810.16
T. Green: 1758.68
P.Mahomes: 1758.31
B.Purdy: 1752.57
R. Gannon: 1736.03
P.Rivers: 1730.38
T.Tagovailoa: 1728.37
